# Linear Analysis of Bernoulli Frames

This notebook demonstrates how to use **pyaxisvm** to build and analyze 3d frames. We will build a scalable model, consisting of a grid of Bernoulli beams, and perform a linear static analysis and a linear vibration analysis. We will visualize the results as tables using `Plotly`.

## Input Data

Python has a waste number of third party packages, that might save you dozens of time. Now we are going to use the `PolyMesh` library to create a grid of lines. The following lines contain all input parameters of the calculations. After modifying these, the rest should work.

In [13]:
# the size of the grid
Lx, Ly, Lz = 1., 0.5, 0.5

# these control subdivision
# set these higher if you want more lines in the same space
nx, ny, nz = 1, 1, 1  

# data of a CHS section
D = 0.1  # outer diameter of the tube in m
t = 0.05  # thickness of the tube in m

# value of the concentrated force at the free end in kN
# and the number of load cases to generate
F = 1.0 
nCase = 15  


### Mesh Creation

We are ready to create our mesh with the help of the `PolyMesh` library:

In [14]:
from polymesh.space import StandardFrame, PointCloud
from polymesh.grid import gridH8 as grid
from polymesh.topo.tr import H8_to_L2
import numpy as np

# mesh
gridparams = {
    'size': (Lx, Ly, Lz),
    'shape': (nx, ny, nz),
    'origo': (0, 0, 0),
    'start': 0
}

# create a grid of hexagonals
coords, topo = grid(**gridparams)

# turn the hexagonal mesh into lines
coords, topo = H8_to_L2(coords, topo)

GlobalFrame = StandardFrame(dim=3)

# define a pointcloud
points = PointCloud(coords, frame=GlobalFrame).centralize()

# set the origo to the center of the free end
dx = - np.array([points[:, 0].min(), 0., 0.])
points.move(dx)

# get the coordinates of the vectors
coords = points.show()


## AxisVM

In [15]:
from axisvm.com.client import start_AxisVM
axvm = start_AxisVM(visible=True, daemon=True)
axvm


IAxisVMApplication,Information
AxisVM Platform,64 bit
AxisVM Version,16 r1s-hf1 En 10
Type Library Version,16 100


In [16]:
import axisvm.com.tlb as axtlb

# create new model
modelId = axvm.Models.New()
axm = axvm.Models[modelId]

# material
ndc, material = axtlb.ndcEuroCode, "S 235"
axm.Settings.NationalDesignCode = ndc

# cross section
MaterialIndex = axm.Materials.AddFromCatalog(ndc, material)
CrossSectionIndex = axm.CrossSections.AddCircleHollow('S1', D, t)

# crate nodes
fnc = axm.Nodes.Add
list(map(lambda c: fnc(*c), coords))

# create lines
fnc = axm.Lines.Add
GeomType = axtlb.lgtStraightLine
list(map(lambda x: fnc(x[0], x[1], GeomType), topo + 1))

# set material and cross section
LineAttr = axtlb.RLineAttr(
    LineType=axtlb.ltBeam,
    MaterialIndex=MaterialIndex,
    StartCrossSectionIndex=CrossSectionIndex,
    EndCrossSectionIndex=CrossSectionIndex
)
lineIDs = [i+1 for i in range(axm.Lines.Count)]
attributes = [LineAttr for _ in range(axm.Lines.Count)]
axm.Lines.BulkSetAttr(lineIDs, attributes)

# essential boundary conditions
spring = axtlb.RStiffnesses(x=1e12, y=1e12, z=1e12, xx=1e12, yy=1e12, zz=1e12)
RNonLinearity = axtlb.RNonLinearity(
    x=axtlb.lnlTensionAndCompression,
    y=axtlb.lnlTensionAndCompression,
    z=axtlb.lnlTensionAndCompression,
    xx=axtlb.lnlTensionAndCompression,
    yy=axtlb.lnlTensionAndCompression,
    zz=axtlb.lnlTensionAndCompression
)
RResistances = axtlb.RResistances(x=0, y=0, z=0, xx=0, yy=0, zz=0)
ebcinds = np.where(coords[:, 0] < 1e-12)[0]
for i in ebcinds:
    axm.NodalSupports.AddNodalGlobal(spring, RNonLinearity, RResistances, i+1)

# natural boundary conditions
load_cases = {}
LoadCaseType = axtlb.lctStandard
inds = np.where(coords[:, 0] > Lx - 1e-12)[0] + 1
axm.BeginUpdate()
for case in range(nCase):
    name = 'LC{}'.format(case+1)
    lcid = axm.LoadCases.Add(name, LoadCaseType)
    pid = np.random.choice(inds)
    Fx, Fy, Fz = 0, 0, -F
    force = axtlb.RLoadNodalForce(
        LoadCaseId=lcid,
        NodeId=pid,
        Fx=Fx, Fy=Fy, Fz=Fz,
        Mx=0., My=0., Mz=0.,
        ReferenceId=0
    )
    axm.Loads.AddNodalForce(force)
    load_cases[lcid] = dict(name=name, id=case)
axm.EndUpdate()

fpath = 'console_H8_L2.axs'
axm.SaveToFile(fpath, False)
axm


IAxisVMModel,Information
N Nodes,8
N Lines,12
N Members,12
N Surfaces,0
N Domains,0


## Linear Static Analysis

We can perform a linear elastic static analysis with the following command:

In [17]:
axm.Calculation.LinearAnalysis(axtlb.cuiNoUserInteractionWithAutoCorrectNoShow)

1

The following block fetches degree of freedom solution:

In [26]:
nIDs = [i+1 for i in range(axm.Nodes.Count)]
disps = axm.Results.Displacements
disps.DisplacementSystem = axtlb.dsGlobal
disps.LoadLevelOrModeShapeOrTimeStep = 1
dofsol = np.zeros((axm.Nodes.Count, 3, nCase))
def f_ex(i): return disps.NodalDisplacementByLoadCaseId(i)[0].ex
def f_ey(i): return disps.NodalDisplacementByLoadCaseId(i)[0].ey
def f_ez(i): return disps.NodalDisplacementByLoadCaseId(i)[0].ez
for lcid, lcdata in load_cases.items():
    lcname = lcdata['name']
    disps.LoadCaseId = lcid
    dataid = lcdata['id']
    dofsol[:, 0, dataid] = np.array(list(map(f_ex, nIDs)))
    dofsol[:, 1, dataid] = np.array(list(map(f_ey, nIDs)))
    dofsol[:, 2, dataid] = np.array(list(map(f_ez, nIDs)))
dofsol[np.where(np.abs(dofsol) < 0.00001)] = 0.0

We can use `Plotly` to visualize our results:

In [27]:
import plotly.graph_objects as go

Font = go.table.header.Font

header=dict(
    values=['Ux', 'Uy', 'Uz'], 
    fill_color='lightskyblue',
    font = Font(size=18))

cells = dict(
    values=[dofsol[:, i, 0] for i in range(3)],
    fill_color='lavender',
    )

fig = go.Figure(data=[go.Table(header=header, cells=cells)])
fig.show()

## Vibration Analysis

In [7]:
options = axtlb.RVibration(
    LoadCase = 1,
    Iterations = 30,
    ModeShapes = 15,
    EigenValueConvergence = 1e-3,
    EigenVectorConvergence = 1e-3,
    MassControl = axtlb.mcMassesOnly,
    ConvertLoadsToMasses = False,
    ConcentratedMasses = False,
    ConvertConcentratedMassesToLoads = False,
    ElementMasses = True,
    MassComponentX = True,
    MassComponentY = True,
    MassComponentZ = True,
    ConvertSlabsToDiaphragms = False,
    MassMatrixType = axtlb.mtConsistent,
    IncreasedSupportStiffness = False,
    MassesTakenIntoAccount=axtlb.mtiaAll
)
interaction = axtlb.cuiNoUserInteractionWithAutoCorrectNoShow
axm.Calculation.Vibration(options, interaction)

 1]

In [29]:
freks, _ = axm.Results.GetAllFrequencies(axtlb.atLinearVibration, 1)

In [9]:
records, _ = axm.Results.GetAllModalMassfactors(axtlb.atLinearVibration, 1)
records = np.array([[r.x, r.y, r.z] for r in records])
records[np.where(records < 0.001)] = 0.0
records *= 100

In [28]:
import plotly.graph_objects as go

Font = go.table.header.Font

header=dict(
    values=['w', 'mx', 'my', 'mz'], 
    fill_color='lightskyblue',
    font = Font(size=18))

cells = dict(
    values=[freks,] + [records[:, i] for i in range(3)],
    fill_color='lavender',
    )

fig = go.Figure(data=[go.Table(header=header, cells=cells)])
fig.show()